# Six-bar mechanism balancing

In [1]:
from BetaShF_population import *
from BetaShM_population import *
import numpy as np
import time
from differential_evolution import differential_evolution

assert ShF([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]) == 1.0
assert ShM([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]) == 1.0
np.random.seed(1)

: 

: 

## Util

In [ ]:
def cleanData(samples, fitness, forces, moments):
    filterF = forces < 1
    filterM = moments < 1
    f = np.logical_and(filterF, filterM)
    print(f.shape)
    return samples[f], fitness[f], forces[f], moments[f]

In [ ]:
def saveData(samples, fitness, forces, moments):
    now = str(int(time.time()))
    np.savetxt(now + "Population.txt", samples)
    np.savetxt(now + "Fitness.txt", fitness)
    np.savetxt(now + "ShForces.txt", forces)
    np.savetxt(now + "ShMoments.txt", moments)
    return now

In [ ]:
def logSample(now, sample, fitness, force, moment):
    def appendToFile(name, text): 
        with open(name, "a") as f:
            f.write(text + '\n')
    s = ""
    for x in sample: s += str(x) + " "
    appendToFile(now + "Population.txt", s)
    appendToFile(now + "Fitness.txt", str(fitness))
    appendToFile(now + "ShForces.txt", str(force))
    appendToFile(now + "ShMoments.txt", str(moment))

## Problem definition

#### Constraints

$$ -0.16m \leq x_{cn}, y_{cn} \leq 0.16m $$

$$ 0.005m \leq t_{cn} \leq 0.04m $$

#### Objective function

$$
f(x) = \alpha ShF(x) + (1 - \alpha) ShM(x)
$$
$$
\therefore \alpha = \frac{f(x) - ShM(x)}{ShF(x) - ShM(x)}
$$

In [ ]:
def objective(s, ShF, ShM, alpha):
    return (alpha)*ShF(s) + (1 - alpha)*ShM(s)

## Sample generation

In [ ]:
#nSamples, nGen = 300, 500 # Tune here
nSamples, nGen = 1, 500 # Tune here

nVariables = 15
nWeights = nVariables // 3
samples = np.zeros((nSamples, nVariables))
bounds = np.zeros((nVariables, 2))
fitness = np.zeros((nSamples))
shForces = np.zeros((nSamples))
shMoments = np.zeros((nSamples))

### Differential Evolution

#### Define boundaries

In [ ]:
for v in range(nVariables):
    bounds[v] = [0.005, 0.04] if (v + 1) % 3 == 0 else [-0.16, 0.16]
print(bounds)

In [ ]:
eTimeDE, goodSols = 0, 0
startTime = str(int(time.time()))
print("Saving data with the prefix:", startTime)
for s in range(nSamples):
    alpha = np.random.normal(0.5, 0.20) # To get values near 0.5
    alpha = np.min([1, np.max([0, alpha])]) # Box constraints [0, 1]
    start = time.perf_counter()
    r = differential_evolution(objective, bounds, popSize = 50, nMax = nGen, args = (ShF, ShM, alpha), cR = 0.7)
    #r = sp.optimize.differential_evolution(objective, bounds, args = (ShF, ShM, alpha), maxiter = nGen, popsize = 50, polish = False)
    #r = cnsg_differential_evolution(objective, bounds, args = (ShF, ShM, alpha), MaxGenerations = nGen, popsize = 50)
    end = time.perf_counter()
    sample, fitness, force, moment = r['x'], r["fun"], ShF(r['x']), ShM(r['x'])
    #sample, fitness, force, moment = r.x, r.fun, ShF(r.x), ShM(r.x)
    #sample, fitness, force, moment = r, objective(r, ShF, ShM, alpha), ShF(r), ShM(r)
    if force < 1 and moment < 1:
        goodSols += 1
        logSample(startTime, sample, fitness, force, moment)
    print("Sample ", s, ":\tshF: ", force, "\tshM: ", moment, "\talpha = ", alpha, sep = "")
    eTimeDE += (end - start) #Time in seconds
if nSamples: eTimeDE /= nSamples

In [ ]:
print("Average time of execution:", eTimeDE, "seconds. It was run", nSamples, "times.")

In [ ]:
print("Went from ", nSamples, " samples to ", goodSols, " after cleaning data", sep = "")